In [1]:
#import libraries
import sqlite3
from urllib.request import urlretrieve
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
import seaborn as sns
import os
import requests as req
import json
import time
import random
from riotwatcher import LolWatcher, ApiError

In [2]:
#Set our variables
api_key = 'RGAPI-a02bf935-a784-455f-80f3-bc0e85d01ba0'
watcher = LolWatcher(api_key)
my_region = 'na1'

In [3]:
#lets test the API 
me = watcher.summoner.by_name(my_region, 'Doublelift')
print(me)

{'id': 'Xxz7tMwFwICmvbNWTd5Ts-fdzn0YTMPSzBbYSusnPUMmy5c', 'accountId': '3o-WQejau7Mv8GPGFaNYU_Aro_YgN2NqLO1gi3nfsaKr-Q', 'puuid': 'nVctAvZgKZG9IwOPO-7p-T-VdoGP1VoyvAEmH9KGxiCN3lgb1zR36q-amdJGvKwhUoHeEafnj4mA8Q', 'name': 'Doublelift', 'profileIconId': 4568, 'revisionDate': 1603503287000, 'summonerLevel': 208}


In [4]:
#collect the data for all available seasons 
url_2020 = 'https://oracleselixir-downloadable-match-data.s3-us-west-2.amazonaws.com/2020_LoL_esports_match_data_from_OraclesElixir_20201031.csv'
url_2019 = 'https://oracleselixir-downloadable-match-data.s3-us-west-2.amazonaws.com/2019_LoL_esports_match_data_from_OraclesElixir_20201031.csv'
url_2018 = 'https://oracleselixir-downloadable-match-data.s3-us-west-2.amazonaws.com/2018_LoL_esports_match_data_from_OraclesElixir_20201031.csv'
url_2017 = 'https://oracleselixir-downloadable-match-data.s3-us-west-2.amazonaws.com/2017_LoL_esports_match_data_from_OraclesElixir_20201031.csv'
url_2016 = 'https://oracleselixir-downloadable-match-data.s3-us-west-2.amazonaws.com/2016_LoL_esports_match_data_from_OraclesElixir_20201031.csv'
url_2015 = 'https://oracleselixir-downloadable-match-data.s3-us-west-2.amazonaws.com/2015_LoL_esports_match_data_from_OraclesElixir_20201031.csv'
url_2014 = 'https://oracleselixir-downloadable-match-data.s3-us-west-2.amazonaws.com/2014_LoL_esports_match_data_from_OraclesElixir_20201031.csv'

In [5]:
fullfilename_2020 = os.path.join('/Users/dankang/Desktop/SpringBoard/Capstone2/Capstone2/data/external', '2020-LoL-matches.csv')
urlretrieve(url_2020, fullfilename_2020)

fullfilename_2019 = os.path.join('/Users/dankang/Desktop/SpringBoard/Capstone2/Capstone2/data/external', '2019-LoL-matches.csv')
urlretrieve(url_2019, fullfilename_2019)

fullfilename_2018 = os.path.join('/Users/dankang/Desktop/SpringBoard/Capstone2/Capstone2/data/external', '2018-LoL-matches.csv')
urlretrieve(url_2018, fullfilename_2018)

fullfilename_2017 = os.path.join('/Users/dankang/Desktop/SpringBoard/Capstone2/Capstone2/data/external', '2017-LoL-matches.csv')
urlretrieve(url_2017, fullfilename_2017)

fullfilename_2016 = os.path.join('/Users/dankang/Desktop/SpringBoard/Capstone2/Capstone2/data/external', '2016-LoL-matches.csv')
urlretrieve(url_2016, fullfilename_2016)

fullfilename_2015 = os.path.join('/Users/dankang/Desktop/SpringBoard/Capstone2/Capstone2/data/external', '2015-LoL-matches.csv')
urlretrieve(url_2015, fullfilename_2015)

fullfilename_2014 = os.path.join('/Users/dankang/Desktop/SpringBoard/Capstone2/Capstone2/data/external', '2014-LoL-matches.csv')
urlretrieve(url_2014, fullfilename_2014)

('/Users/dankang/Desktop/SpringBoard/Capstone2/Capstone2/data/external/2014-LoL-matches.csv',
 <http.client.HTTPMessage at 0x7f86d0849a90>)

In [6]:
#save to a dataframe
game_data_2020 = pd.read_csv(fullfilename_2020, low_memory=False)
game_data_2019 = pd.read_csv(fullfilename_2019, low_memory=False)
game_data_2018 = pd.read_csv(fullfilename_2018, low_memory=False)
game_data_2017 = pd.read_csv(fullfilename_2017, low_memory=False)
game_data_2016 = pd.read_csv(fullfilename_2016, low_memory=False)
game_data_2015 = pd.read_csv(fullfilename_2015, low_memory=False)
game_data_2014 = pd.read_csv(fullfilename_2014, low_memory=False)

In [7]:
game_data_2020['season'] = '2020'
game_data_2019['season'] = '2019'
game_data_2018['season'] = '2018'
game_data_2017['season'] = '2017'
game_data_2016['season'] = '2016'
game_data_2015['season'] = '2015'
game_data_2014['season'] = '2014'

In [8]:
cols = list(game_data_2020)

In [9]:
print(cols)

['gameid', 'datacompleteness', 'url', 'league', 'year', 'split', 'playoffs', 'date', 'game', 'patch', 'playerid', 'side', 'position', 'player', 'team', 'champion', 'ban1', 'ban2', 'ban3', 'ban4', 'ban5', 'gamelength', 'result', 'kills', 'deaths', 'assists', 'teamkills', 'teamdeaths', 'doublekills', 'triplekills', 'quadrakills', 'pentakills', 'firstblood', 'firstbloodkill', 'firstbloodassist', 'firstbloodvictim', 'team kpm', 'ckpm', 'firstdragon', 'dragons', 'opp_dragons', 'elementaldrakes', 'opp_elementaldrakes', 'infernals', 'mountains', 'clouds', 'oceans', 'dragons (type unknown)', 'elders', 'opp_elders', 'firstherald', 'heralds', 'opp_heralds', 'firstbaron', 'barons', 'opp_barons', 'firsttower', 'towers', 'opp_towers', 'firstmidtower', 'firsttothreetowers', 'inhibitors', 'opp_inhibitors', 'damagetochampions', 'dpm', 'damageshare', 'damagetakenperminute', 'damagemitigatedperminute', 'wardsplaced', 'wpm', 'wardskilled', 'wcpm', 'controlwardsbought', 'visionscore', 'vspm', 'totalgold',

In [10]:
game_data_2019 = game_data_2019.rename(columns=dict(zip(game_data_2019.columns,cols)))
game_data_2018 = game_data_2018.rename(columns=dict(zip(game_data_2018.columns,cols)))
game_data_2017 = game_data_2017.rename(columns=dict(zip(game_data_2017.columns,cols)))
game_data_2016 = game_data_2016.rename(columns=dict(zip(game_data_2016.columns,cols)))
game_data_2015 = game_data_2015.rename(columns=dict(zip(game_data_2015.columns,cols)))
game_data_2014 = game_data_2014.rename(columns=dict(zip(game_data_2014.columns,cols)))

In [11]:
frames = [game_data_2020, game_data_2019, game_data_2018, game_data_2017, game_data_2016, game_data_2015, game_data_2014]

In [12]:
#concatenate the dataframes
game_data = pd.concat(frames)

In [13]:
game_data.dtypes

gameid               object
datacompleteness     object
url                  object
league               object
year                  int64
                     ...   
opp_csat15          float64
golddiffat15        float64
xpdiffat15          float64
csdiffat15          float64
season               object
Length: 106, dtype: object

In [14]:
#lets take a look at the data
game_data.info()
game_data.dtypes

<class 'pandas.core.frame.DataFrame'>
Int64Index: 369911 entries, 0 to 11015
Columns: 106 entries, gameid to season
dtypes: float64(77), int64(12), object(17)
memory usage: 302.0+ MB


gameid               object
datacompleteness     object
url                  object
league               object
year                  int64
                     ...   
opp_csat15          float64
golddiffat15        float64
xpdiffat15          float64
csdiffat15          float64
season               object
Length: 106, dtype: object

In [15]:
print(game_data.shape)

(369911, 106)


In [16]:
game_data.head(1000)

,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,...,goldat15,xpat15,csat15,opp_goldat15,opp_xpat15,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,season
0,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,2020,NaN,0,2020-01-03 07:33:26,1,9.24,...,4888.0,7368.0,131.0,4723.0,7202.0,118.0,165.0,166.0,13.0,2020
1,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,2020,NaN,0,2020-01-03 07:33:26,1,9.24,...,4385.0,4817.0,91.0,4784.0,4667.0,98.0,-399.0,150.0,-7.0,2020
2,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,2020,NaN,0,2020-01-03 07:33:26,1,9.24,...,4809.0,6275.0,129.0,5218.0,8112.0,140.0,-409.0,-1837.0,-11.0,2020
3,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,2020,NaN,0,2020-01-03 07:33:26,1,9.24,...,4915.0,4959.0,136.0,4864.0,5360.0,135.0,51.0,-401.0,1.0,2020
4,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,2020,NaN,0,2020-01-03 07:33:26,1,9.24,...,2956.0,3800.0,20.0,3189.0,3543.0,28.0,-233.0,257.0,-8.0,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,ESPORTSTMNT02/1270857,complete,https://matchhistory.na.leagueoflegends.com/en...,LCS,2020,Spring,0,2020-01-26 20:35:10,1,10.01,...,21300.0,26287.0,465.0,24165.0,27241.0,496.0,-2865.0,-954.0,-31.0,2020
996,ESPORTSTMNT02/1270866,complete,https://matchhistory.na.leagueoflegends.com/en...,LCS,2020,Spring,0,2020-01-26 22:02:06,1,10.01,...,4978.0,6992.0,108.0,4733.0,6892.0,114.0,245.0,100.0,-6.0,2020
997,ESPORTSTMNT02/1270866,complete,https://matchhistory.na.leagueoflegends.com/en...,LCS,2020,Spring,0,2020-01-26 22:02:06,1,10.01,...,5150.0,4942.0,82.0,4869.0,5125.0,92.0,281.0,-183.0,-10.0,2020
998,ESPORTSTMNT02/1270866,complete,https://matchhistory.na.leagueoflegends.com/en...,LCS,2020,Spring,0,2020-01-26 22:02:06,1,10.01,...,5174.0,7324.0,125.0,5405.0,7832.0,147.0,-231.0,-508.0,-22.0,2020


From here we can see there are 369,911 entries and 104 columns / features

In [17]:
#check for null values
game_data.isnull().values.any()

True

There are null entries, but rather than address them now we will address them a bit later

In [18]:
#lets isolate the number of league entries
game_data.league.unique()

array(['KeSPA', 'LPL', 'LFL', 'LEC', 'LCS.A', 'CBLOL', 'LCS', 'OPL',
       'VCS', 'TCL', 'TRA', 'BRCC', 'LCK', 'CK', 'LJL', 'LLA', 'LCL',
       'PCS', 'EM', 'LDL', 'OCS', 'Riot', 'MSC', 'WCS', 'LMS', 'MSI',
       'DC', 'EU LCS', 'LLN', 'CLS', 'NA LCS', 'TCS', 'CIS CL', 'GPL',
       'EU CS', 'LSPL', 'NA CS', 'TPL', 'IEM', 'LTC'], dtype=object)

We want to eventually isolate the LCS. We note that the LCS.A is the Academy/minor league, so we ignore that entry later on



In [19]:
#make sure year is datetime object -- fix this
game_data['year'] = game_data['year'].astype('category')
game_data.dtypes

gameid                object
datacompleteness      object
url                   object
league                object
year                category
                      ...   
opp_csat15           float64
golddiffat15         float64
xpdiffat15           float64
csdiffat15           float64
season                object
Length: 106, dtype: object

In [20]:
#do the same for playerid, game, and playoffs: make sure they are categories
game_data['playerid'] = game_data['playerid'].astype('category')
game_data['game'] = game_data['game'].astype('category')
game_data['playoffs'] = game_data['playoffs'].astype('category')
game_data['season'] = game_data['season'].astype('category')
game_data['date'] = pd.to_datetime(game_data['date'])

In [21]:
#lets take a look at the first ten entries of our dataframe
game_data.head(10)

,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,...,goldat15,xpat15,csat15,opp_goldat15,opp_xpat15,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,season
0,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,2020,NaN,0,2020-01-03 07:33:26,1,9.24,...,4888.0,7368.0,131.0,4723.0,7202.0,118.0,165.0,166.0,13.0,2020
1,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,2020,NaN,0,2020-01-03 07:33:26,1,9.24,...,4385.0,4817.0,91.0,4784.0,4667.0,98.0,-399.0,150.0,-7.0,2020
2,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,2020,NaN,0,2020-01-03 07:33:26,1,9.24,...,4809.0,6275.0,129.0,5218.0,8112.0,140.0,-409.0,-1837.0,-11.0,2020
3,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,2020,NaN,0,2020-01-03 07:33:26,1,9.24,...,4915.0,4959.0,136.0,4864.0,5360.0,135.0,51.0,-401.0,1.0,2020
4,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,2020,NaN,0,2020-01-03 07:33:26,1,9.24,...,2956.0,3800.0,20.0,3189.0,3543.0,28.0,-233.0,257.0,-8.0,2020
5,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,2020,NaN,0,2020-01-03 07:33:26,1,9.24,...,4723.0,7202.0,118.0,4888.0,7368.0,131.0,-165.0,-166.0,-13.0,2020
6,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,2020,NaN,0,2020-01-03 07:33:26,1,9.24,...,4784.0,4667.0,98.0,4385.0,4817.0,91.0,399.0,-150.0,7.0,2020
7,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,2020,NaN,0,2020-01-03 07:33:26,1,9.24,...,5218.0,8112.0,140.0,4809.0,6275.0,129.0,409.0,1837.0,11.0,2020
8,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,2020,NaN,0,2020-01-03 07:33:26,1,9.24,...,4864.0,5360.0,135.0,4915.0,4959.0,136.0,-51.0,401.0,-1.0,2020
9,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,2020,NaN,0,2020-01-03 07:33:26,1,9.24,...,3189.0,3543.0,28.0,2956.0,3800.0,20.0,233.0,-257.0,8.0,2020


In [22]:
game_data = game_data.drop(['year'], axis=1)

In [23]:
game_data.head()

,gameid,datacompleteness,url,league,split,playoffs,date,game,patch,playerid,...,goldat15,xpat15,csat15,opp_goldat15,opp_xpat15,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,season
0,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,NaN,0,2020-01-03 07:33:26,1,9.24,1,...,4888.0,7368.0,131.0,4723.0,7202.0,118.0,165.0,166.0,13.0,2020
1,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,NaN,0,2020-01-03 07:33:26,1,9.24,2,...,4385.0,4817.0,91.0,4784.0,4667.0,98.0,-399.0,150.0,-7.0,2020
2,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,NaN,0,2020-01-03 07:33:26,1,9.24,3,...,4809.0,6275.0,129.0,5218.0,8112.0,140.0,-409.0,-1837.0,-11.0,2020
3,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,NaN,0,2020-01-03 07:33:26,1,9.24,4,...,4915.0,4959.0,136.0,4864.0,5360.0,135.0,51.0,-401.0,1.0,2020
4,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,NaN,0,2020-01-03 07:33:26,1,9.24,5,...,2956.0,3800.0,20.0,3189.0,3543.0,28.0,-233.0,257.0,-8.0,2020


There are more than two entries per game, despite there only being 2 teams. This is because our dataframe currently aggregates match statistics for individual players -- we care only about the team. In this notebook we will isolate the team entries.

In [24]:
#lets isolate all the seasons and put it into an array
seasons = game_data['season'].unique()

In [25]:
print(seasons)

['2020', '2019', '2018', '2017', '2016', '2015', '2014']
Categories (7, object): ['2020', '2019', '2018', '2017', '2016', '2015', '2014']


In [26]:
game_data[game_data.gameid == 'ESPORTSTMNT03/1241318'].T

,0,1,2,3,4,5,6,7,8,9,10,11
gameid,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318
datacompleteness,complete,complete,complete,complete,complete,complete,complete,complete,complete,complete,complete,complete
url,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...
league,KeSPA,KeSPA,KeSPA,KeSPA,KeSPA,KeSPA,KeSPA,KeSPA,KeSPA,KeSPA,KeSPA,KeSPA
split,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
opp_csat15,118,98,140,135,28,131,91,129,136,20,519,507
golddiffat15,165,-399,-409,51,-233,-165,399,409,-51,233,-825,825
xpdiffat15,166,150,-1837,-401,257,-166,-150,1837,401,-257,-1665,1665
csdiffat15,13,-7,-11,1,-8,-13,7,11,-1,8,-12,12


In fact, there are 12 entries for the first game listed: 5 team members, one team entry (and 2 teams. Let's explore the datatypes of the columns

In [27]:
game_data.select_dtypes(include='object')

,gameid,datacompleteness,url,league,split,side,position,player,team,champion,ban1,ban2,ban3,ban4,ban5
0,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,NaN,Blue,top,Summit,SANDBOX Gaming,Rumble,LeBlanc,Irelia,Rek'Sai,Yasuo,Renekton
1,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,NaN,Blue,jng,OnFleek,SANDBOX Gaming,Elise,LeBlanc,Irelia,Rek'Sai,Yasuo,Renekton
2,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,NaN,Blue,mid,FATE,SANDBOX Gaming,Qiyana,LeBlanc,Irelia,Rek'Sai,Yasuo,Renekton
3,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,NaN,Blue,bot,Leo,SANDBOX Gaming,Miss Fortune,LeBlanc,Irelia,Rek'Sai,Yasuo,Renekton
4,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,NaN,Blue,sup,GorillA,SANDBOX Gaming,Nautilus,LeBlanc,Irelia,Rek'Sai,Yasuo,Renekton
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11011,FRA1TMNT1/430152,complete,http://matchhistory.na.leagueoflegends.com/en/...,EU LCS,Spring,Red,mid,Pepiinero,Giants Gaming,Ahri,Zed,Irelia,LeBlanc,NaN,NaN
11012,FRA1TMNT1/430152,complete,http://matchhistory.na.leagueoflegends.com/en/...,EU LCS,Spring,Red,bot,Adryh,Giants Gaming,Corki,Zed,Irelia,LeBlanc,NaN,NaN
11013,FRA1TMNT1/430152,complete,http://matchhistory.na.leagueoflegends.com/en/...,EU LCS,Spring,Red,sup,Rydle,Giants Gaming,Morgana,Zed,Irelia,LeBlanc,NaN,NaN
11014,FRA1TMNT1/430152,complete,http://matchhistory.na.leagueoflegends.com/en/...,EU LCS,Spring,Blue,team,NaN,Reason Gaming,NaN,Syndra,Xerath,Pantheon,NaN,NaN


In [28]:
game_data[game_data.gameid == 'ESPORTSTMNT03/1241318'].T.head(26)

,0,1,2,3,4,5,6,7,8,9,10,11
gameid,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318,ESPORTSTMNT03/1241318
datacompleteness,complete,complete,complete,complete,complete,complete,complete,complete,complete,complete,complete,complete
url,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...,http://matchhistory.na.leagueoflegends.com/en/...
league,KeSPA,KeSPA,KeSPA,KeSPA,KeSPA,KeSPA,KeSPA,KeSPA,KeSPA,KeSPA,KeSPA,KeSPA
split,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
playoffs,0,0,0,0,0,0,0,0,0,0,0,0
date,2020-01-03 07:33:26,2020-01-03 07:33:26,2020-01-03 07:33:26,2020-01-03 07:33:26,2020-01-03 07:33:26,2020-01-03 07:33:26,2020-01-03 07:33:26,2020-01-03 07:33:26,2020-01-03 07:33:26,2020-01-03 07:33:26,2020-01-03 07:33:26,2020-01-03 07:33:26
game,1,1,1,1,1,1,1,1,1,1,1,1
patch,9.24,9.24,9.24,9.24,9.24,9.24,9.24,9.24,9.24,9.24,9.24,9.24
playerid,1,2,3,4,5,6,7,8,9,10,100,200


Lets now isolate our league of interest

In [29]:
#isolate LCS
array = ['NA LCS', 'LCS']
LCS_data = game_data.loc[game_data['league'].isin(array)]
print(LCS_data.head())
LCS_data.info()

                    gameid datacompleteness  \
864  ESPORTSTMNT02/1270555         complete   
865  ESPORTSTMNT02/1270555         complete   
866  ESPORTSTMNT02/1270555         complete   
867  ESPORTSTMNT02/1270555         complete   
868  ESPORTSTMNT02/1270555         complete   

                                                   url league   split  \
864  https://matchhistory.na.leagueoflegends.com/en...    LCS  Spring   
865  https://matchhistory.na.leagueoflegends.com/en...    LCS  Spring   
866  https://matchhistory.na.leagueoflegends.com/en...    LCS  Spring   
867  https://matchhistory.na.leagueoflegends.com/en...    LCS  Spring   
868  https://matchhistory.na.leagueoflegends.com/en...    LCS  Spring   

    playoffs                date game  patch playerid  ... goldat15  xpat15  \
864        0 2020-01-25 21:37:53    1  10.01        1  ...   5193.0  8018.0   
865        0 2020-01-25 21:37:53    1  10.01        2  ...   4189.0  4560.0   
866        0 2020-01-25 21:37:53    1  10

In [30]:
#check for null columns
null_columns = LCS_data.columns[LCS_data.isnull().any()]
LCS_data[null_columns].isnull().sum().sort_values(ascending=False)

dragons (type unknown)    26762
firstherald               23476
opp_heralds               23476
heralds                   23476
elementaldrakes           22320
opp_elementaldrakes       22320
infernals                 22320
mountains                 22320
oceans                    22320
elders                    22320
opp_elders                22320
clouds                    22320
firstbaron                22310
gspd                      22310
firstdragon               22310
dragons                   22310
opp_dragons               22310
firsttothreetowers        22310
barons                    22310
firstmidtower             22310
opp_towers                22310
towers                    22310
firsttower                22310
opp_barons                22310
ban5                      11658
ban4                      11598
champion                   4462
player                     4462
firstbloodkill             4462
total cs                   4462
firstbloodassist           4462
firstblo

We note that the features with 22310+ null entries are team statistics -- meaning there are entries for each team but not for the players, as teams get credited with those game accomplishments. This means when we isolate the team entries these should no longer show up as null.

In [31]:
#find the rows with null game ids
LCS_data[LCS_data["gameid"].isnull()]

,gameid,datacompleteness,url,league,split,playoffs,date,game,patch,playerid,...,goldat15,xpat15,csat15,opp_goldat15,opp_xpat15,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,season


We find that there are no missing game IDs in our LCS dataframe, which is good (and verified above as well)

In [32]:
LCS_data[LCS_data['player'].isnull()].head(10000)

,gameid,datacompleteness,url,league,split,playoffs,date,game,patch,playerid,...,goldat15,xpat15,csat15,opp_goldat15,opp_xpat15,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,season
874,ESPORTSTMNT02/1270555,complete,https://matchhistory.na.leagueoflegends.com/en...,LCS,Spring,0,2020-01-25 21:37:53,1,10.01,100,...,22949.0,28643.0,485.0,22995.0,27534.0,481.0,-46.0,1109.0,4.0,2020
875,ESPORTSTMNT02/1270555,complete,https://matchhistory.na.leagueoflegends.com/en...,LCS,Spring,0,2020-01-25 21:37:53,1,10.01,200,...,22995.0,27534.0,481.0,22949.0,28643.0,485.0,46.0,-1109.0,-4.0,2020
886,ESPORTSTMNT02/1270576,complete,https://matchhistory.na.leagueoflegends.com/en...,LCS,Spring,0,2020-01-25 22:32:55,1,10.01,100,...,23196.0,26241.0,486.0,24778.0,29394.0,522.0,-1582.0,-3153.0,-36.0,2020
887,ESPORTSTMNT02/1270576,complete,https://matchhistory.na.leagueoflegends.com/en...,LCS,Spring,0,2020-01-25 22:32:55,1,10.01,200,...,24778.0,29394.0,522.0,23196.0,26241.0,486.0,1582.0,3153.0,36.0,2020
898,ESPORTSTMNT02/1270592,complete,https://matchhistory.na.leagueoflegends.com/en...,LCS,Spring,0,2020-01-25 23:39:30,1,10.01,100,...,24058.0,26322.0,461.0,22600.0,26940.0,461.0,1458.0,-618.0,0.0,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10811,FRA1TMNT1/410317,complete,http://matchhistory.na.leagueoflegends.com/en/...,NA LCS,Spring,0,2014-12-14 21:11:00,3,4.20,200,...,21173.0,26442.0,356.0,20287.0,26613.0,411.0,886.0,-171.0,-55.0,2014
10822,FRA1TMNT1/410318,complete,http://matchhistory.na.leagueoflegends.com/en/...,NA LCS,Spring,0,2014-12-14 22:01:34,4,4.20,100,...,21231.0,24742.0,331.0,21534.0,23899.0,376.0,-303.0,843.0,-45.0,2014
10823,FRA1TMNT1/410318,complete,http://matchhistory.na.leagueoflegends.com/en/...,NA LCS,Spring,0,2014-12-14 22:01:34,4,4.20,200,...,21534.0,23899.0,376.0,21231.0,24742.0,331.0,303.0,-843.0,45.0,2014
10834,FRA1TMNT1/410321,complete,http://matchhistory.na.leagueoflegends.com/en/...,NA LCS,Spring,0,2014-12-14 22:55:49,5,4.20,100,...,20949.0,27644.0,412.0,20210.0,26715.0,414.0,739.0,929.0,-2.0,2014


These entries likely correspond to entries for the "team" position, for which there are null stats on the individual player features... so lets filter further to confirm

In [33]:
#get numerical statistics
LCS_data.describe().T

,count,mean,std,min,25%,50%,75%,max
patch,25812.0,7.016588,1.827604,4.03,6.02,7.05,8.15,10.16
gamelength,26772.0,2166.428507,451.533709,1100.00,1830.00,2111.00,2422.00,4837.00
result,26772.0,0.500000,0.500009,0.00,0.00,0.50,1.00,1.00
kills,26772.0,4.069102,5.017262,0.00,1.00,2.00,5.00,41.00
deaths,26772.0,4.077245,4.799628,0.00,1.00,3.00,5.00,41.00
...,...,...,...,...,...,...,...,...
opp_xpat15,26772.0,9524.933064,8652.229948,1660.00,4939.00,6058.00,7573.00,34463.00
opp_csat15,26772.0,154.950769,147.118122,0.00,69.00,121.00,145.00,629.00
golddiffat15,26772.0,0.000000,1228.433575,-11037.00,-525.25,0.00,525.25,11037.00
xpdiffat15,26772.0,0.000000,1026.067751,-7719.00,-512.00,0.00,512.00,7719.00


In [34]:
#lets look for missing entries:
missing = pd.concat([LCS_data.isnull().sum(), 100 * LCS_data.isnull().mean()], axis=1)
missing.columns=['count', '%']
missing.sort_values(by='count', ascending=False)
missing.head()

,count,%
gameid,0,0.0
datacompleteness,0,0.0
url,0,0.0
league,0,0.0
split,0,0.0


In [35]:
#lets find the number of unique games ### come back to this
print(LCS_data['gameid'].nunique())

2231


2231 unique games

In [36]:
#isolate team performances
team = LCS_data.position == 'team'
LCS_data_by_team = LCS_data[team]

In [37]:
LCS_data_by_team['team'].value_counts()

Team SoloMid            497
Cloud9                  453
Counter Logic Gaming    435
Team Liquid             401
Dignitas                286
FlyQuest                261
Echo Fox                246
Immortals               210
Phoenix1                160
Team EnVyUs             156
100 Thieves             145
Golden Guardians        130
Evil Geniuses           123
Clutch Gaming            99
Team Impulse             83
Team Coast               77
OpTic Gaming             76
Team Curse               74
NRG Esports              69
Apex Gaming              53
Team 8                   47
Gravity                  45
LMQ                      41
compLexity Gaming        33
XDG Gaming               31
Team Dragon Knights      31
Enemy                    25
Gold Coin United         22
Winterfox                22
Renegades                22
Team Fusion              21
eUnited                  17
Final Five               14
Curse Academy            14
compLexity.Black          6
Team Liquid Academy 

In [38]:
#lets filter out any teams that are not official teams
official_teams = ['Team SoloMid', 'Cloud9', 'Team Liquid', 'Immortals', 'Golden Guardians', 'FlyQuest', 'Evil Geniuses', 'Dignitas', 'Counter Logic Gaming', '100 Thieves']

In [39]:
LCS_data_official = LCS_data_by_team.loc[LCS_data_by_team['team'].isin(official_teams)]

In [40]:
LCS_data_official.head()

,gameid,datacompleteness,url,league,split,playoffs,date,game,patch,playerid,...,goldat15,xpat15,csat15,opp_goldat15,opp_xpat15,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,season
874,ESPORTSTMNT02/1270555,complete,https://matchhistory.na.leagueoflegends.com/en...,LCS,Spring,0,2020-01-25 21:37:53,1,10.01,100,...,22949.0,28643.0,485.0,22995.0,27534.0,481.0,-46.0,1109.0,4.0,2020
875,ESPORTSTMNT02/1270555,complete,https://matchhistory.na.leagueoflegends.com/en...,LCS,Spring,0,2020-01-25 21:37:53,1,10.01,200,...,22995.0,27534.0,481.0,22949.0,28643.0,485.0,46.0,-1109.0,-4.0,2020
886,ESPORTSTMNT02/1270576,complete,https://matchhistory.na.leagueoflegends.com/en...,LCS,Spring,0,2020-01-25 22:32:55,1,10.01,100,...,23196.0,26241.0,486.0,24778.0,29394.0,522.0,-1582.0,-3153.0,-36.0,2020
887,ESPORTSTMNT02/1270576,complete,https://matchhistory.na.leagueoflegends.com/en...,LCS,Spring,0,2020-01-25 22:32:55,1,10.01,200,...,24778.0,29394.0,522.0,23196.0,26241.0,486.0,1582.0,3153.0,36.0,2020
898,ESPORTSTMNT02/1270592,complete,https://matchhistory.na.leagueoflegends.com/en...,LCS,Spring,0,2020-01-25 23:39:30,1,10.01,100,...,24058.0,26322.0,461.0,22600.0,26940.0,461.0,1458.0,-618.0,0.0,2020


In [41]:
LCS_data_official['team_name'] = LCS_data_official['team']

/Users/dankang/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [42]:
#lets isolate all the seasons and teams and put it into an array
seasons = LCS_data_official['season'].unique()
teams = LCS_data_official['team'].unique()
print(seasons)
print(teams)

['2020', '2019', '2018', '2017', '2016', '2015', '2014']
Categories (7, object): ['2020', '2019', '2018', '2017', '2016', '2015', '2014']
['Cloud9' 'Team Liquid' 'Counter Logic Gaming' 'Dignitas' '100 Thieves'
 'Golden Guardians' 'Immortals' 'FlyQuest' 'Evil Geniuses' 'Team SoloMid']


In [43]:
iterables = [seasons, teams]

In [44]:
pd.MultiIndex.from_product(iterables, names=['season', 'team'])

MultiIndex([('2020',               'Cloud9'),
            ('2020',          'Team Liquid'),
            ('2020', 'Counter Logic Gaming'),
            ('2020',             'Dignitas'),
            ('2020',          '100 Thieves'),
            ('2020',     'Golden Guardians'),
            ('2020',            'Immortals'),
            ('2020',             'FlyQuest'),
            ('2020',        'Evil Geniuses'),
            ('2020',         'Team SoloMid'),
            ('2019',               'Cloud9'),
            ('2019',          'Team Liquid'),
            ('2019', 'Counter Logic Gaming'),
            ('2019',             'Dignitas'),
            ('2019',          '100 Thieves'),
            ('2019',     'Golden Guardians'),
            ('2019',            'Immortals'),
            ('2019',             'FlyQuest'),
            ('2019',        'Evil Geniuses'),
            ('2019',         'Team SoloMid'),
            ('2018',               'Cloud9'),
            ('2018',          'Tea

In [45]:
LCS_data_reindexed = LCS_data_official.set_index(['season', 'team'])

In [46]:
LCS_data_reindexed.head(1000)

gameid datacompleteness  \
season team                                                           
2020   Cloud9                ESPORTSTMNT02/1270555         complete   
       Team Liquid           ESPORTSTMNT02/1270555         complete   
       Counter Logic Gaming  ESPORTSTMNT02/1270576         complete   
       Dignitas              ESPORTSTMNT02/1270576         complete   
       100 Thieves           ESPORTSTMNT02/1270592         complete   
...                                            ...              ...   
2018   Cloud9                     TRLH1/1002510212         complete   
       Team Liquid                TRLH1/1002510212         complete   
       Team Liquid                TRLH1/1002510216         complete   
       Cloud9                     TRLH1/1002510216         complete   
       Team SoloMid               TRLH1/1002520202         complete   

                                                                           url  \
season team                                                                      
2020   Cloud9                https://matchhistory.na.leagueoflegends.com/en...   
       Team Liquid           https://matchhistory.na.leagueoflegends.com/en...   
       Counter Logic Gaming  https://matchhistory.na.leagueoflegends.com/en...   
       Dignitas              https://matchhistory.na.leagueoflegends.com/en...   
       100 Thieves           https://matchhistory.na.leagueoflegends.com/en...   
...                                                                        ...   
2018   Cloud9                https://matchhistory.na.leagueoflegends.com/en...   
       Team Liquid           https://matchhistory.na.leagueoflegends.com/en...   
       Team Liquid           https://matchhistory.na.leagueoflegends.com/en...   
       Cloud9                https://matchhistory.na.leagueoflegends.com/en...   
       Team SoloMid          https://matchhistory.na.leagueoflegends.com/en...   

                             league   split playoffs                date game  \
season team                                                                     
2020   Cloud9                   LCS  Spring        0 2020-01-25 21:37:53    1   
       Team Liquid              LCS  Spring        0 2020-01-25 21:37:53    1   
       Counter Logic Gaming     LCS  Spring        0 2020-01-25 22:32:55    1   
       Dignitas                 LCS  Spring        0 2020-01-25 22:32:55    1   
       100 Thieves              LCS  Spring        0 2020-01-25 23:39:30    1   
...                             ...     ...      ...                 ...  ...   
2018   Cloud9                NA LCS  Spring        1 2018-03-24 21:58:16    2   
       Team Liquid           NA LCS  Spring        1 2018-03-24 21:58:16    2   
       Team Liquid           NA LCS  Spring        1 2018-03-24 22:53:05    3   
       Cloud9                NA LCS  Spring        1 2018-03-24 22:53:05    3   
       Team SoloMid          NA LCS  Spring        1 2018-03-25 18:50:34    1   

                             patch playerid  ... goldat15   xpat15 csat15  \
season team                                  ...                            
2020   Cloud9                10.01      100  ...  22949.0  28643.0  485.0   
       Team Liquid           10.01      200  ...  22995.0  27534.0  481.0   
       Counter Logic Gaming  10.01      100  ...  23196.0  26241.0  486.0   
       Dignitas              10.01      200  ...  24778.0  29394.0  522.0   
       100 Thieves           10.01      100  ...  24058.0  26322.0  461.0   
...                            ...      ...  ...      ...      ...    ...   
2018   Cloud9                 8.05      100  ...  25389.0  30282.0  527.0   
       Team Liquid            8.05      200  ...  23599.0  29149.0  505.0   
       Team Liquid            8.05      100  ...  24717.0  30224.0  566.0   
       Cloud9                 8.05      200  ...  23425.0  30401.0  575.0   
       Team SoloMid           8.05      100  ...  25313.0  31570.0  586.0  

In [47]:
LCS_data_official_2020 = LCS_data_reindexed.filter(like='2020', axis=0)
LCS_data_official_2019 = LCS_data_reindexed.filter(like='2019', axis=0)
LCS_data_official_2018 = LCS_data_reindexed.filter(like='2018', axis=0)
LCS_data_official_2017 = LCS_data_reindexed.filter(like='2017', axis=0)
LCS_data_official_2016 = LCS_data_reindexed.filter(like='2016', axis=0)
LCS_data_official_2015 = LCS_data_reindexed.filter(like='2015', axis=0)
LCS_data_official_2014 = LCS_data_reindexed.filter(like='2014', axis=0)

In [48]:
#lets store the game counts in a dictionary for later use
game_counts_2020 = LCS_data_official_2020['team_name'].value_counts().to_dict()
game_counts_2019 = LCS_data_official_2019['team_name'].value_counts().to_dict()
game_counts_2018 = LCS_data_official_2018['team_name'].value_counts().to_dict()
game_counts_2017 = LCS_data_official_2017['team_name'].value_counts().to_dict()
game_counts_2016 = LCS_data_official_2016['team_name'].value_counts().to_dict()
game_counts_2015 = LCS_data_official_2015['team_name'].value_counts().to_dict()
game_counts_2014 = LCS_data_official_2014['team_name'].value_counts().to_dict()

In [49]:
print(game_counts_2020)
print(game_counts_2019)
print(game_counts_2018)
print(game_counts_2017)
print(game_counts_2016)
print(game_counts_2015)
print(game_counts_2014)

{'FlyQuest': 75, 'Team SoloMid': 71, 'Evil Geniuses': 60, 'Cloud9': 57, 'Golden Guardians': 51, '100 Thieves': 49, 'Team Liquid': 49, 'Dignitas': 42, 'Counter Logic Gaming': 37, 'Immortals': 37}
{'Team Liquid': 54, 'Team SoloMid': 54, 'Cloud9': 51, 'Counter Logic Gaming': 48, 'FlyQuest': 45, 'Golden Guardians': 43, '100 Thieves': 37}
{'100 Thieves': 59, 'Team SoloMid': 59, 'Team Liquid': 55, 'Cloud9': 49, 'FlyQuest': 41, 'Counter Logic Gaming': 36, 'Golden Guardians': 36}
{'Dignitas': 109, 'Team Liquid': 108, 'Counter Logic Gaming': 106, 'Team SoloMid': 104, 'Cloud9': 101, 'FlyQuest': 100, 'Immortals': 96}
{'Counter Logic Gaming': 86, 'Cloud9': 79, 'Team SoloMid': 78, 'Immortals': 77, 'Team Liquid': 76, 'Dignitas': 21}
{'Team Liquid': 59, 'Team SoloMid': 55, 'Counter Logic Gaming': 47, 'Cloud9': 47, 'Dignitas': 44}
{'Team SoloMid': 76, 'Counter Logic Gaming': 75, 'Dignitas': 70, 'Cloud9': 69, 'Evil Geniuses': 63}


In [50]:
LCS_data_reindexed[LCS_data_reindexed['player'].isnull()].head(10000)

gameid datacompleteness  \
season team                                                           
2020   Cloud9                ESPORTSTMNT02/1270555         complete   
       Team Liquid           ESPORTSTMNT02/1270555         complete   
       Counter Logic Gaming  ESPORTSTMNT02/1270576         complete   
       Dignitas              ESPORTSTMNT02/1270576         complete   
       100 Thieves           ESPORTSTMNT02/1270592         complete   
...                                            ...              ...   
2014   Counter Logic Gaming       TRLH1/1000340110         complete   
       Counter Logic Gaming       TRLH1/1000340112         complete   
       Evil Geniuses              TRLH1/1000350123         complete   
       Evil Geniuses              TRLH1/1000350130         complete   
       Evil Geniuses              TRLH1/1000350131         complete   

                                                                           url  \
season team                                                                      
2020   Cloud9                https://matchhistory.na.leagueoflegends.com/en...   
       Team Liquid           https://matchhistory.na.leagueoflegends.com/en...   
       Counter Logic Gaming  https://matchhistory.na.leagueoflegends.com/en...   
       Dignitas              https://matchhistory.na.leagueoflegends.com/en...   
       100 Thieves           https://matchhistory.na.leagueoflegends.com/en...   
...                                                                        ...   
2014   Counter Logic Gaming  http://matchhistory.na.leagueoflegends.com/en/...   
       Counter Logic Gaming  http://matchhistory.na.leagueoflegends.com/en/...   
       Evil Geniuses         http://matchhistory.na.leagueoflegends.com/en/...   
       Evil Geniuses         http://matchhistory.na.leagueoflegends.com/en/...   
       Evil Geniuses         http://matchhistory.na.leagueoflegends.com/en/...   

                             league   split playoffs                date game  \
season team                                                                     
2020   Cloud9                   LCS  Spring        0 2020-01-25 21:37:53    1   
       Team Liquid              LCS  Spring        0 2020-01-25 21:37:53    1   
       Counter Logic Gaming     LCS  Spring        0 2020-01-25 22:32:55    1   
       Dignitas                 LCS  Spring        0 2020-01-25 22:32:55    1   
       100 Thieves              LCS  Spring        0 2020-01-25 23:39:30    1   
...                             ...     ...      ...                 ...  ...   
2014   Counter Logic Gaming  NA LCS  Spring        0 2014-09-09 23:17:28    4   
       Counter Logic Gaming  NA LCS  Spring        0 2014-09-10 00:10:32    5   
       Evil Geniuses         NA LCS  Spring        0 2014-09-10 21:05:07    1   
       Evil Geniuses         NA LCS  Spring        0 2014-09-10 21:59:24    2   
       Evil Geniuses         NA LCS  Spring        0 2014-09-10 22:58:26    3   

                             patch playerid  ... goldat15   xpat15 csat15  \
season team                                  ...                            
2020   Cloud9                10.01      100  ...  22949.0  28643.0  485.0   
       Team Liquid           10.01      200  ...  22995.0  27534.0  481.0   
       Counter Logic Gaming  10.01      100  ...  23196.0  26241.0  486.0   
       Dignitas              10.01      200  ...  24778.0  29394.0  522.0   
       100 Thieves           10.01      100  ...  24058.0  26322.0  461.0   
...                            ...      ...  ...      ...      ...    ...   
2014   Counter Logic Gaming   4.14      200  ...  24343.0  30137.0  461.0   
       Counter Logic Gaming   4.14      100  ...  24237.0  31354.0  487.0   
       Evil Geniuses          4.14      100  ...  25176.0  30569.0  451.0   
       Evil Geniuses          4.14      200  ...  21474.0  29400.0  480.0   
       Evil Geniuses          4.14      100  ...  21940.0  27712.0  409.0  

In [51]:
LCS_data_reindexed.head(10000)

gameid datacompleteness  \
season team                                                           
2020   Cloud9                ESPORTSTMNT02/1270555         complete   
       Team Liquid           ESPORTSTMNT02/1270555         complete   
       Counter Logic Gaming  ESPORTSTMNT02/1270576         complete   
       Dignitas              ESPORTSTMNT02/1270576         complete   
       100 Thieves           ESPORTSTMNT02/1270592         complete   
...                                            ...              ...   
2014   Counter Logic Gaming       TRLH1/1000340110         complete   
       Counter Logic Gaming       TRLH1/1000340112         complete   
       Evil Geniuses              TRLH1/1000350123         complete   
       Evil Geniuses              TRLH1/1000350130         complete   
       Evil Geniuses              TRLH1/1000350131         complete   

                                                                           url  \
season team                                                                      
2020   Cloud9                https://matchhistory.na.leagueoflegends.com/en...   
       Team Liquid           https://matchhistory.na.leagueoflegends.com/en...   
       Counter Logic Gaming  https://matchhistory.na.leagueoflegends.com/en...   
       Dignitas              https://matchhistory.na.leagueoflegends.com/en...   
       100 Thieves           https://matchhistory.na.leagueoflegends.com/en...   
...                                                                        ...   
2014   Counter Logic Gaming  http://matchhistory.na.leagueoflegends.com/en/...   
       Counter Logic Gaming  http://matchhistory.na.leagueoflegends.com/en/...   
       Evil Geniuses         http://matchhistory.na.leagueoflegends.com/en/...   
       Evil Geniuses         http://matchhistory.na.leagueoflegends.com/en/...   
       Evil Geniuses         http://matchhistory.na.leagueoflegends.com/en/...   

                             league   split playoffs                date game  \
season team                                                                     
2020   Cloud9                   LCS  Spring        0 2020-01-25 21:37:53    1   
       Team Liquid              LCS  Spring        0 2020-01-25 21:37:53    1   
       Counter Logic Gaming     LCS  Spring        0 2020-01-25 22:32:55    1   
       Dignitas                 LCS  Spring        0 2020-01-25 22:32:55    1   
       100 Thieves              LCS  Spring        0 2020-01-25 23:39:30    1   
...                             ...     ...      ...                 ...  ...   
2014   Counter Logic Gaming  NA LCS  Spring        0 2014-09-09 23:17:28    4   
       Counter Logic Gaming  NA LCS  Spring        0 2014-09-10 00:10:32    5   
       Evil Geniuses         NA LCS  Spring        0 2014-09-10 21:05:07    1   
       Evil Geniuses         NA LCS  Spring        0 2014-09-10 21:59:24    2   
       Evil Geniuses         NA LCS  Spring        0 2014-09-10 22:58:26    3   

                             patch playerid  ... goldat15   xpat15 csat15  \
season team                                  ...                            
2020   Cloud9                10.01      100  ...  22949.0  28643.0  485.0   
       Team Liquid           10.01      200  ...  22995.0  27534.0  481.0   
       Counter Logic Gaming  10.01      100  ...  23196.0  26241.0  486.0   
       Dignitas              10.01      200  ...  24778.0  29394.0  522.0   
       100 Thieves           10.01      100  ...  24058.0  26322.0  461.0   
...                            ...      ...  ...      ...      ...    ...   
2014   Counter Logic Gaming   4.14      200  ...  24343.0  30137.0  461.0   
       Counter Logic Gaming   4.14      100  ...  24237.0  31354.0  487.0   
       Evil Geniuses          4.14      100  ...  25176.0  30569.0  451.0   
       Evil Geniuses          4.14      200  ...  21474.0  29400.0  480.0   
       Evil Geniuses          4.14      100  ...  21940.0  27712.0  409.0  

In [52]:
#isolate stats for the first game of the first match in the dataframe
first_game = LCS_data_reindexed.gameid == 'ESPORTSTMNT02/1270555'
LCS_data_first_game = LCS_data_reindexed[first_game]
LCS_data_first_game.head(500)

gameid datacompleteness  \
season team                                                  
2020   Cloud9       ESPORTSTMNT02/1270555         complete   
       Team Liquid  ESPORTSTMNT02/1270555         complete   

                                                                  url league  \
season team                                                                    
2020   Cloud9       https://matchhistory.na.leagueoflegends.com/en...    LCS   
       Team Liquid  https://matchhistory.na.leagueoflegends.com/en...    LCS   

                     split playoffs                date game  patch playerid  \
season team                                                                    
2020   Cloud9       Spring        0 2020-01-25 21:37:53    1  10.01      100   
       Team Liquid  Spring        0 2020-01-25 21:37:53    1  10.01      200   

                    ... goldat15   xpat15 csat15 opp_goldat15 opp_xpat15  \
season team         ...                                                    
2020   Cloud9       ...  22949.0  28643.0  485.0      22995.0    27534.0   
       Team Liquid  ...  22995.0  27534.0  481.0      22949.0    28643.0   

                   opp_csat15 golddiffat15 xpdiffat15 csdiffat15    team_name  
season team                                                                    
2020   Cloud9           481.0        -46.0     1109.0        4.0       Cloud9  
       Team Liquid      485.0         46.0    -1109.0       -4.0  Team Liquid  

[2 rows x 104 columns]

In [53]:
#select all numeric columns
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
newdf = LCS_data_first_game.select_dtypes(include=numerics)
all_stats = newdf.columns

In [54]:
team_totals = LCS_data_reindexed.groupby(level=[0,1])[all_stats].sum().round(2)
print(team_totals.head(100))
print(team_totals.info())

                              patch  gamelength  result   kills  deaths  \
season team                                                               
2014   100 Thieves              NaN         NaN     NaN     NaN     NaN   
       Cloud9                200.39    151548.0    52.0  1025.0   695.0   
       Counter Logic Gaming  225.05    157623.0    40.0   946.0   907.0   
       Dignitas              204.44    161507.0    32.0   824.0   879.0   
       Evil Geniuses         175.64    141069.0    25.0   771.0   872.0   
...                             ...         ...     ...     ...     ...   
2020   FlyQuest              757.22    155511.0    43.0   955.0   825.0   
       Golden Guardians      514.98    106331.0    23.0   486.0   612.0   
       Immortals             373.04     80059.0    12.0   298.0   462.0   
       Team Liquid           495.06    106392.0    29.0   560.0   465.0   
       Team SoloMid          717.58    151006.0    41.0   874.0   734.0   

                        

In [55]:
team_totals['gamesplayed'] = pd.Series('0')
team_totals.loc[:,'gamesplayed'] = 0

In [57]:
team_totals_2020 = team_totals.filter(like='2020', axis=0)
team_totals_2019 = team_totals.filter(like='2019', axis=0)
team_totals_2018 = team_totals.filter(like='2018', axis=0)
team_totals_2017 = team_totals.filter(like='2017', axis=0)
team_totals_2016 = team_totals.filter(like='2016', axis=0)
team_totals_2015 = team_totals.filter(like='2015', axis=0)
team_totals_2014 = team_totals.filter(like='2014', axis=0)

In [58]:
team_totals_2020 = team_totals_2020.reset_index(level='team')
team_totals_2019 = team_totals_2019.reset_index(level='team')
team_totals_2018 = team_totals_2018.reset_index(level='team')
team_totals_2017 = team_totals_2017.reset_index(level='team')
team_totals_2016 = team_totals_2016.reset_index(level='team')
team_totals_2015 = team_totals_2015.reset_index(level='team')
team_totals_2014 = team_totals_2014.reset_index(level='team')

In [59]:
team_totals_2020['gamesplayed'] = team_totals_2020["team"].apply(lambda x: game_counts_2020.get(x))
team_totals_2019['gamesplayed'] = team_totals_2019["team"].apply(lambda x: game_counts_2019.get(x))
team_totals_2018['gamesplayed'] = team_totals_2018["team"].apply(lambda x: game_counts_2018.get(x))
team_totals_2017['gamesplayed'] = team_totals_2017["team"].apply(lambda x: game_counts_2017.get(x))
team_totals_2016['gamesplayed'] = team_totals_2016["team"].apply(lambda x: game_counts_2016.get(x))
team_totals_2015['gamesplayed'] = team_totals_2015["team"].apply(lambda x: game_counts_2015.get(x))
team_totals_2014['gamesplayed'] = team_totals_2014["team"].apply(lambda x: game_counts_2014.get(x))

In [77]:
new_frames = [team_totals_2020, team_totals_2019, team_totals_2018, team_totals_2017, team_totals_2016, team_totals_2015, team_totals_2014]

In [78]:
df = pd.concat(new_frames)

In [79]:
df.head(1000)

,team,patch,gamelength,result,kills,deaths,assists,teamkills,teamdeaths,doublekills,...,goldat15,xpat15,csat15,opp_goldat15,opp_xpat15,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,gamesplayed
season,,,,,,,,,,,,,,,,,,,,,
2020,100 Thieves,494.06,98775.0,20.0,534.0,625.0,1213.0,534.0,625.0,59.0,...,1159872.0,1402835.0,24271.0,1196839.0,1427357.0,24539.0,-36967.0,-24522.0,-268.0,49.0
2020,Cloud9,575.34,106591.0,44.0,892.0,528.0,2073.0,892.0,528.0,105.0,...,1457462.0,1699932.0,29083.0,1339319.0,1616509.0,27782.0,118143.0,83423.0,1301.0,57.0
2020,Counter Logic Gaming,373.13,76646.0,8.0,360.0,532.0,903.0,360.0,532.0,27.0,...,869714.0,1069385.0,18978.0,926688.0,1108046.0,19352.0,-56974.0,-38661.0,-374.0,37.0
2020,Dignitas,423.73,86579.0,15.0,390.0,464.0,931.0,390.0,464.0,41.0,...,1007052.0,1219211.0,21575.0,1028798.0,1237821.0,22083.0,-21746.0,-18610.0,-508.0,42.0
2020,Evil Geniuses,605.52,121008.0,29.0,670.0,777.0,1571.0,670.0,777.0,63.0,...,1436032.0,1727591.0,29739.0,1465996.0,1747335.0,30474.0,-29964.0,-19744.0,-735.0,60.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014,FlyQuest,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014,Golden Guardians,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014,Immortals,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [81]:
df = df.reset_index(level='season')

In [82]:
df = df.set_index(['season', 'team'])

In [83]:
df.head(1000)

patch  gamelength  result   kills  deaths  \
season team                                                               
2020   100 Thieves           494.06     98775.0    20.0   534.0   625.0   
       Cloud9                575.34    106591.0    44.0   892.0   528.0   
       Counter Logic Gaming  373.13     76646.0     8.0   360.0   532.0   
       Dignitas              423.73     86579.0    15.0   390.0   464.0   
       Evil Geniuses         605.52    121008.0    29.0   670.0   777.0   
...                             ...         ...     ...     ...     ...   
2014   FlyQuest                 NaN         NaN     NaN     NaN     NaN   
       Golden Guardians         NaN         NaN     NaN     NaN     NaN   
       Immortals                NaN         NaN     NaN     NaN     NaN   
       Team Liquid              NaN         NaN     NaN     NaN     NaN   
       Team SoloMid          229.28    172375.0    49.0  1066.0   781.0   

                             assists  teamkills  teamdeaths  doublekills  \
season team                                                                
2020   100 Thieves            1213.0      534.0       625.0         59.0   
       Cloud9                 2073.0      892.0       528.0        105.0   
       Counter Logic Gaming    903.0      360.0       532.0         27.0   
       Dignitas                931.0      390.0       464.0         41.0   
       Evil Geniuses          1571.0      670.0       777.0         63.0   
...                              ...        ...         ...          ...   
2014   FlyQuest                  NaN        NaN         NaN          NaN   
       Golden Guardians          NaN        NaN         NaN          NaN   
       Immortals                 NaN        NaN         NaN          NaN   
       Team Liquid               NaN        NaN         NaN          NaN   
       Team SoloMid           2630.0     1066.0       781.0        121.0   

                             triplekills  ...   goldat15     xpat15   csat15  \
season team                               ...                                  
2020   100 Thieves                   4.0  ...  1159872.0  1402835.0  24271.0   
       Cloud9                       20.0  ...  1457462.0  1699932.0  29083.0   
       Counter Logic Gaming          6.0  ...   869714.0  1069385.0  18978.0   
       Dignitas                      7.0  ...  1007052.0  1219211.0  21575.0   
       Evil Geniuses                 5.0  ...  1436032.0  1727591.0  29739.0   
...                                  ...  ...        ...        ...      ...   
2014   FlyQuest                      NaN  ...        NaN        NaN      NaN   
       Golden Guardians              NaN  ...        NaN        NaN      NaN   
       Immortals                     NaN  ...        NaN        NaN      NaN   
       Team Liquid                   NaN  ...        NaN        NaN      NaN   
       Team SoloMid                 24.0  ...  1639431.0  2165650.0  33291.0   

                             opp_goldat15  opp_xpat15  opp_csat15  \
season team                                                         
2020   100 Thieves              1196839.0   1427357.0     24539.0   
       Cloud9                   1339319.0   1616509.0     27782.0   
       Counter Logic Gaming      926688.0   1108046.0     19352.0   
       Dignitas                 1028798.0   1237821.0     22083.0   
       Evil Geniuses            1465996.0   1747335.0     30474.0   
...                                   ...         ...         ...   
2014   FlyQuest                       NaN         NaN         NaN   
       Golden Guardians               NaN         NaN         NaN   
       Immortals                      NaN         NaN         NaN   
       Team Liquid                    NaN         NaN         NaN   
       Team SoloMid             1600720.0   2103462.0     32592.0   

                             golddiffat15  xpdiffat15  csdiffat15  gamesplayed  
season team                                  

In [84]:
df = df.dropna()

In [85]:
df.head(1000)

patch  gamelength  result   kills  deaths  \
season team                                                               
2020   100 Thieves           494.06     98775.0    20.0   534.0   625.0   
       Cloud9                575.34    106591.0    44.0   892.0   528.0   
       Counter Logic Gaming  373.13     76646.0     8.0   360.0   532.0   
       Dignitas              423.73     86579.0    15.0   390.0   464.0   
       Evil Geniuses         605.52    121008.0    29.0   670.0   777.0   
       FlyQuest              757.22    155511.0    43.0   955.0   825.0   
       Golden Guardians      514.98    106331.0    23.0   486.0   612.0   
       Immortals             373.04     80059.0    12.0   298.0   462.0   
       Team Liquid           495.06    106392.0    29.0   560.0   465.0   
       Team SoloMid          717.58    151006.0    41.0   874.0   734.0   
2019   100 Thieves           335.92     77827.0    12.0   320.0   419.0   
       Cloud9                463.57    101493.0    34.0   730.0   536.0   
       Counter Logic Gaming  436.57    100032.0    25.0   494.0   523.0   
       FlyQuest              408.31     92939.0    18.0   426.0   474.0   
       Golden Guardians      390.27     89142.0    19.0   463.0   526.0   
       Team Liquid           490.76    108299.0    40.0   736.0   498.0   
       Team SoloMid          490.22    115461.0    32.0   647.0   577.0   
2018   100 Thieves           477.55    132272.0    34.0   641.0   572.0   
       Cloud9                396.41    106292.0    26.0   543.0   538.0   
       Counter Logic Gaming  290.88     80978.0    14.0   384.0   398.0   
       FlyQuest              331.66     88449.0    16.0   410.0   498.0   
       Golden Guardians      290.88     77826.0     9.0   352.0   421.0   
       Team Liquid           444.60    109674.0    39.0   620.0   454.0   
       Team SoloMid          477.88    128578.0    33.0   617.0   538.0   
2017   Cloud9                715.03    218648.0    65.0  1458.0  1126.0   
       Counter Logic Gaming  751.07    230569.0    57.0  1564.0  1506.0   
       Dignitas              772.57    237727.0    55.0  1287.0  1328.0   
       FlyQuest              707.56    214296.0    44.0  1235.0  1508.0   
       Immortals             680.02    208812.0    55.0  1365.0  1263.0   
       Team Liquid           764.59    230421.0    42.0  1309.0  1426.0   
       Team SoloMid          736.25    222771.0    73.0  1532.0  1198.0   
2016   Cloud9                481.94    169754.0    48.0  1107.0   843.0   
       Counter Logic Gaming  524.49    180752.0    49.0  1054.0  1054.0   
       Dignitas              126.62     46931.0     4.0   204.0   269.0   
       Immortals             469.54    155968.0    58.0  1187.0   918.0   
       Team Liquid           462.98    155723.0    38.0   961.0   911.0   
       Team SoloMid          475.23    159167.0    58.0  1116.0   775.0   
2015   Cloud9                238.26    103636.0    24.0   551.0   576.0   
       Counter Logic Gaming  238.74    103616.0    31.0   717.0   584.0   
       Dignitas              223.26    100514.0    19.0   514.0   546.0   
       Team Liquid           299.62    139888.0    36.0   844.0   698.0   
       Team SoloMid          279.56    130050.0    36.0   769.0   604.0   
2014   Cloud9                200.39    151548.0    52.0  1025.0   695.0   
       Counter Logic Gaming  225.05    157623.0    40.0   946.0   907.0   
       Dignitas              204.44    161507.0    32.0   824.0   879.0   
       Evil Geniuses         175.64    141069.0    25.0   771.0   872.0   
       Team SoloMid          229.28    172375.0    49.0  1066.0   781.0   

                             assists  teamkills  teamdeaths  doublekills  \
season team                                                                
2020   100 Thieves            1213.0      534.0       625.0         59.0   
       Cloud9                 2073.0      892.0       528.0        105.0   
       Counter Logic Gaming    903.0      360.0   

In [86]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 47 entries, ('2020', '100 Thieves') to ('2014', 'Team SoloMid')
Data columns (total 86 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   patch                     47 non-null     float64
 1   gamelength                47 non-null     float64
 2   result                    47 non-null     float64
 3   kills                     47 non-null     float64
 4   deaths                    47 non-null     float64
 5   assists                   47 non-null     float64
 6   teamkills                 47 non-null     float64
 7   teamdeaths                47 non-null     float64
 8   doublekills               47 non-null     float64
 9   triplekills               47 non-null     float64
 10  quadrakills               47 non-null     float64
 11  pentakills                47 non-null     float64
 12  firstblood                47 non-null     float64
 13  firstbloodkill         

Let's move the cells so that the game totals are first, and the results second, so we can compute wins and losses. 

In [87]:
# get a list of columns
cols = list(df)
# move the column to head of list using index, pop and insert
cols.insert(0, cols.pop(cols.index('gamesplayed')))
cols

['gamesplayed',
 'patch',
 'gamelength',
 'result',
 'kills',
 'deaths',
 'assists',
 'teamkills',
 'teamdeaths',
 'doublekills',
 'triplekills',
 'quadrakills',
 'pentakills',
 'firstblood',
 'firstbloodkill',
 'firstbloodassist',
 'firstbloodvictim',
 'team kpm',
 'ckpm',
 'firstdragon',
 'dragons',
 'opp_dragons',
 'elementaldrakes',
 'opp_elementaldrakes',
 'infernals',
 'mountains',
 'clouds',
 'oceans',
 'dragons (type unknown)',
 'elders',
 'opp_elders',
 'firstherald',
 'heralds',
 'opp_heralds',
 'firstbaron',
 'barons',
 'opp_barons',
 'firsttower',
 'towers',
 'opp_towers',
 'firstmidtower',
 'firsttothreetowers',
 'inhibitors',
 'opp_inhibitors',
 'damagetochampions',
 'dpm',
 'damageshare',
 'damagetakenperminute',
 'damagemitigatedperminute',
 'wardsplaced',
 'wpm',
 'wardskilled',
 'wcpm',
 'controlwardsbought',
 'visionscore',
 'vspm',
 'totalgold',
 'earnedgold',
 'earned gpm',
 'earnedgoldshare',
 'goldspent',
 'gspd',
 'total cs',
 'minionkills',
 'monsterkills',
 'm

In [88]:
# use ix to reorder
df = df.loc[:, cols]
df

gamesplayed   patch  gamelength  result   kills  \
season team                                                                    
2020   100 Thieves                  49.0  494.06     98775.0    20.0   534.0   
       Cloud9                       57.0  575.34    106591.0    44.0   892.0   
       Counter Logic Gaming         37.0  373.13     76646.0     8.0   360.0   
       Dignitas                     42.0  423.73     86579.0    15.0   390.0   
       Evil Geniuses                60.0  605.52    121008.0    29.0   670.0   
       FlyQuest                     75.0  757.22    155511.0    43.0   955.0   
       Golden Guardians             51.0  514.98    106331.0    23.0   486.0   
       Immortals                    37.0  373.04     80059.0    12.0   298.0   
       Team Liquid                  49.0  495.06    106392.0    29.0   560.0   
       Team SoloMid                 71.0  717.58    151006.0    41.0   874.0   
2019   100 Thieves                  37.0  335.92     77827.0    12.0   320.0   
       Cloud9                       51.0  463.57    101493.0    34.0   730.0   
       Counter Logic Gaming         48.0  436.57    100032.0    25.0   494.0   
       FlyQuest                     45.0  408.31     92939.0    18.0   426.0   
       Golden Guardians             43.0  390.27     89142.0    19.0   463.0   
       Team Liquid                  54.0  490.76    108299.0    40.0   736.0   
       Team SoloMid                 54.0  490.22    115461.0    32.0   647.0   
2018   100 Thieves                  59.0  477.55    132272.0    34.0   641.0   
       Cloud9                       49.0  396.41    106292.0    26.0   543.0   
       Counter Logic Gaming         36.0  290.88     80978.0    14.0   384.0   
       FlyQuest                     41.0  331.66     88449.0    16.0   410.0   
       Golden Guardians             36.0  290.88     77826.0     9.0   352.0   
       Team Liquid                  55.0  444.60    109674.0    39.0   620.0   
       Team SoloMid                 59.0  477.88    128578.0    33.0   617.0   
2017   Cloud9                      101.0  715.03    218648.0    65.0  1458.0   
       Counter Logic Gaming        106.0  751.07    230569.0    57.0  1564.0   
       Dignitas                    109.0  772.57    237727.0    55.0  1287.0   
       FlyQuest                    100.0  707.56    214296.0    44.0  1235.0   
       Immortals                    96.0  680.02    208812.0    55.0  1365.0   
       Team Liquid                 108.0  764.59    230421.0    42.0  1309.0   
       Team SoloMid                104.0  736.25    222771.0    73.0  1532.0   
2016   Cloud9                       79.0  481.94    169754.0    48.0  1107.0   
       Counter Logic Gaming         86.0  524.49    180752.0    49.0  1054.0   
       Dignitas                     21.0  126.62     46931.0     4.0   204.0   
       Immortals                    77.0  469.54    155968.0    58.0  1187.0   
       Team Liquid                  76.0  462.98    155723.0    38.0   961.0   
       Team SoloMid                 78.0  475.23    159167.0    58.0  1116.0   
2015   Cloud9                       47.0  238.26    103636.0    24.0   551.0   
       Counter Logic Gaming         47.0  238.74    103616.0    31.0   717.0   
       Dignitas                     44.0  223.26    100514.0    19.0   514.0   
       Team Liquid                  59.0  299.62    139888.0    36.0   844.0   
       Team SoloMid                 55.0  279.56    130050.0    36.0   769.0   
2014   Cloud9                       69.0  200.39    151548.0    52.0  1025.0   
       Counter Logic Gaming         75.0  225.05    157623.0    40.0   946.0   
       Dignitas                     70.0  204.44    161507.0    32.0   824.0   
       Evil Geniuses                63.0  175.64    141069.0    25.0   771.0   
       Team SoloMid                 76.0  229.28    172375.0    49.0  1066.0   

                             deaths  assists  teamkills  teamdeaths  \
season team                          

In [89]:
# get a list of columns
cols = list(df)
# move the column to head of list using index, pop and insert
cols.insert(1, cols.pop(cols.index('result')))
df = df.loc[:, cols]
df

gamesplayed  result   patch  gamelength   kills  \
season team                                                                    
2020   100 Thieves                  49.0    20.0  494.06     98775.0   534.0   
       Cloud9                       57.0    44.0  575.34    106591.0   892.0   
       Counter Logic Gaming         37.0     8.0  373.13     76646.0   360.0   
       Dignitas                     42.0    15.0  423.73     86579.0   390.0   
       Evil Geniuses                60.0    29.0  605.52    121008.0   670.0   
       FlyQuest                     75.0    43.0  757.22    155511.0   955.0   
       Golden Guardians             51.0    23.0  514.98    106331.0   486.0   
       Immortals                    37.0    12.0  373.04     80059.0   298.0   
       Team Liquid                  49.0    29.0  495.06    106392.0   560.0   
       Team SoloMid                 71.0    41.0  717.58    151006.0   874.0   
2019   100 Thieves                  37.0    12.0  335.92     77827.0   320.0   
       Cloud9                       51.0    34.0  463.57    101493.0   730.0   
       Counter Logic Gaming         48.0    25.0  436.57    100032.0   494.0   
       FlyQuest                     45.0    18.0  408.31     92939.0   426.0   
       Golden Guardians             43.0    19.0  390.27     89142.0   463.0   
       Team Liquid                  54.0    40.0  490.76    108299.0   736.0   
       Team SoloMid                 54.0    32.0  490.22    115461.0   647.0   
2018   100 Thieves                  59.0    34.0  477.55    132272.0   641.0   
       Cloud9                       49.0    26.0  396.41    106292.0   543.0   
       Counter Logic Gaming         36.0    14.0  290.88     80978.0   384.0   
       FlyQuest                     41.0    16.0  331.66     88449.0   410.0   
       Golden Guardians             36.0     9.0  290.88     77826.0   352.0   
       Team Liquid                  55.0    39.0  444.60    109674.0   620.0   
       Team SoloMid                 59.0    33.0  477.88    128578.0   617.0   
2017   Cloud9                      101.0    65.0  715.03    218648.0  1458.0   
       Counter Logic Gaming        106.0    57.0  751.07    230569.0  1564.0   
       Dignitas                    109.0    55.0  772.57    237727.0  1287.0   
       FlyQuest                    100.0    44.0  707.56    214296.0  1235.0   
       Immortals                    96.0    55.0  680.02    208812.0  1365.0   
       Team Liquid                 108.0    42.0  764.59    230421.0  1309.0   
       Team SoloMid                104.0    73.0  736.25    222771.0  1532.0   
2016   Cloud9                       79.0    48.0  481.94    169754.0  1107.0   
       Counter Logic Gaming         86.0    49.0  524.49    180752.0  1054.0   
       Dignitas                     21.0     4.0  126.62     46931.0   204.0   
       Immortals                    77.0    58.0  469.54    155968.0  1187.0   
       Team Liquid                  76.0    38.0  462.98    155723.0   961.0   
       Team SoloMid                 78.0    58.0  475.23    159167.0  1116.0   
2015   Cloud9                       47.0    24.0  238.26    103636.0   551.0   
       Counter Logic Gaming         47.0    31.0  238.74    103616.0   717.0   
       Dignitas                     44.0    19.0  223.26    100514.0   514.0   
       Team Liquid                  59.0    36.0  299.62    139888.0   844.0   
       Team SoloMid                 55.0    36.0  279.56    130050.0   769.0   
2014   Cloud9                       69.0    52.0  200.39    151548.0  1025.0   
       Counter Logic Gaming         75.0    40.0  225.05    157623.0   946.0   
       Dignitas                     70.0    32.0  204.44    161507.0   824.0   
       Evil Geniuses                63.0    25.0  175.64    141069.0   771.0   
       Team SoloMid                 76.0    49.0  229.28    172375.0  1066.0   

                             deaths  assists  teamkills  teamdeaths  \
season team                          

In [90]:
df.insert(2, 'Losses', 0)

In [91]:
df.head()

gamesplayed  result  Losses   patch  gamelength  \
season team                                                                    
2020   100 Thieves                  49.0    20.0       0  494.06     98775.0   
       Cloud9                       57.0    44.0       0  575.34    106591.0   
       Counter Logic Gaming         37.0     8.0       0  373.13     76646.0   
       Dignitas                     42.0    15.0       0  423.73     86579.0   
       Evil Geniuses                60.0    29.0       0  605.52    121008.0   

                             kills  deaths  assists  teamkills  teamdeaths  \
season team                                                                  
2020   100 Thieves           534.0   625.0   1213.0      534.0       625.0   
       Cloud9                892.0   528.0   2073.0      892.0       528.0   
       Counter Logic Gaming  360.0   532.0    903.0      360.0       532.0   
       Dignitas              390.0   464.0    931.0      390.0       464.0   
       Evil Geniuses         670.0   777.0   1571.0      670.0       777.0   

                             ...  csdiffat10   goldat15     xpat15   csat15  \
season team                  ...                                              
2020   100 Thieves           ...       -99.0  1159872.0  1402835.0  24271.0   
       Cloud9                ...       824.0  1457462.0  1699932.0  29083.0   
       Counter Logic Gaming  ...      -139.0   869714.0  1069385.0  18978.0   
       Dignitas              ...      -321.0  1007052.0  1219211.0  21575.0   
       Evil Geniuses         ...      -499.0  1436032.0  1727591.0  29739.0   

                             opp_goldat15  opp_xpat15  opp_csat15  \
season team                                                         
2020   100 Thieves              1196839.0   1427357.0     24539.0   
       Cloud9                   1339319.0   1616509.0     27782.0   
       Counter Logic Gaming      926688.0   1108046.0     19352.0   
       Dignitas                 1028798.0   1237821.0     22083.0   
       Evil Geniuses            1465996.0   1747335.0     30474.0   

                             golddiffat15  xpdiffat15  csdiffat15  
season team                                                        
2020   100 Thieves               -36967.0    -24522.0      -268.0  
       Cloud9                    118143.0     83423.0      1301.0  
       Counter Logic Gaming      -56974.0    -38661.0      -374.0  
       Dignitas                  -21746.0    -18610.0      -508.0  
       Evil Geniuses             -29964.0    -19744.0      -735.0  

[5 rows x 87 columns]

In [92]:
mapping = {df.columns[1]: 'Wins'}
df = df.rename(columns=mapping)
df.head()

gamesplayed  Wins  Losses   patch  gamelength  \
season team                                                                  
2020   100 Thieves                  49.0  20.0       0  494.06     98775.0   
       Cloud9                       57.0  44.0       0  575.34    106591.0   
       Counter Logic Gaming         37.0   8.0       0  373.13     76646.0   
       Dignitas                     42.0  15.0       0  423.73     86579.0   
       Evil Geniuses                60.0  29.0       0  605.52    121008.0   

                             kills  deaths  assists  teamkills  teamdeaths  \
season team                                                                  
2020   100 Thieves           534.0   625.0   1213.0      534.0       625.0   
       Cloud9                892.0   528.0   2073.0      892.0       528.0   
       Counter Logic Gaming  360.0   532.0    903.0      360.0       532.0   
       Dignitas              390.0   464.0    931.0      390.0       464.0   
       Evil Geniuses         670.0   777.0   1571.0      670.0       777.0   

                             ...  csdiffat10   goldat15     xpat15   csat15  \
season team                  ...                                              
2020   100 Thieves           ...       -99.0  1159872.0  1402835.0  24271.0   
       Cloud9                ...       824.0  1457462.0  1699932.0  29083.0   
       Counter Logic Gaming  ...      -139.0   869714.0  1069385.0  18978.0   
       Dignitas              ...      -321.0  1007052.0  1219211.0  21575.0   
       Evil Geniuses         ...      -499.0  1436032.0  1727591.0  29739.0   

                             opp_goldat15  opp_xpat15  opp_csat15  \
season team                                                         
2020   100 Thieves              1196839.0   1427357.0     24539.0   
       Cloud9                   1339319.0   1616509.0     27782.0   
       Counter Logic Gaming      926688.0   1108046.0     19352.0   
       Dignitas                 1028798.0   1237821.0     22083.0   
       Evil Geniuses            1465996.0   1747335.0     30474.0   

                             golddiffat15  xpdiffat15  csdiffat15  
season team                                                        
2020   100 Thieves               -36967.0    -24522.0      -268.0  
       Cloud9                    118143.0     83423.0      1301.0  
       Counter Logic Gaming      -56974.0    -38661.0      -374.0  
       Dignitas                  -21746.0    -18610.0      -508.0  
       Evil Geniuses             -29964.0    -19744.0      -735.0  

[5 rows x 87 columns]

In [93]:
df.insert(3, 'Win_Percentage', 0)
df.head()

gamesplayed  Wins  Losses  Win_Percentage  \
season team                                                              
2020   100 Thieves                  49.0  20.0       0               0   
       Cloud9                       57.0  44.0       0               0   
       Counter Logic Gaming         37.0   8.0       0               0   
       Dignitas                     42.0  15.0       0               0   
       Evil Geniuses                60.0  29.0       0               0   

                              patch  gamelength  kills  deaths  assists  \
season team                                                               
2020   100 Thieves           494.06     98775.0  534.0   625.0   1213.0   
       Cloud9                575.34    106591.0  892.0   528.0   2073.0   
       Counter Logic Gaming  373.13     76646.0  360.0   532.0    903.0   
       Dignitas              423.73     86579.0  390.0   464.0    931.0   
       Evil Geniuses         605.52    121008.0  670.0   777.0   1571.0   

                             teamkills  ...  csdiffat10   goldat15     xpat15  \
season team                             ...                                     
2020   100 Thieves               534.0  ...       -99.0  1159872.0  1402835.0   
       Cloud9                    892.0  ...       824.0  1457462.0  1699932.0   
       Counter Logic Gaming      360.0  ...      -139.0   869714.0  1069385.0   
       Dignitas                  390.0  ...      -321.0  1007052.0  1219211.0   
       Evil Geniuses             670.0  ...      -499.0  1436032.0  1727591.0   

                              csat15  opp_goldat15  opp_xpat15  opp_csat15  \
season team                                                                  
2020   100 Thieves           24271.0     1196839.0   1427357.0     24539.0   
       Cloud9                29083.0     1339319.0   1616509.0     27782.0   
       Counter Logic Gaming  18978.0      926688.0   1108046.0     19352.0   
       Dignitas              21575.0     1028798.0   1237821.0     22083.0   
       Evil Geniuses         29739.0     1465996.0   1747335.0     30474.0   

                             golddiffat15  xpdiffat15  csdiffat15  
season team                                                        
2020   100 Thieves               -36967.0    -24522.0      -268.0  
       Cloud9                    118143.0     83423.0      1301.0  
       Counter Logic Gaming      -56974.0    -38661.0      -374.0  
       Dignitas                  -21746.0    -18610.0      -508.0  
       Evil Geniuses             -29964.0    -19744.0      -735.0  

[5 rows x 88 columns]

In [94]:
df['Losses'] = df['gamesplayed'] - df['Wins'] 
df.head()

gamesplayed  Wins  Losses  Win_Percentage  \
season team                                                              
2020   100 Thieves                  49.0  20.0    29.0               0   
       Cloud9                       57.0  44.0    13.0               0   
       Counter Logic Gaming         37.0   8.0    29.0               0   
       Dignitas                     42.0  15.0    27.0               0   
       Evil Geniuses                60.0  29.0    31.0               0   

                              patch  gamelength  kills  deaths  assists  \
season team                                                               
2020   100 Thieves           494.06     98775.0  534.0   625.0   1213.0   
       Cloud9                575.34    106591.0  892.0   528.0   2073.0   
       Counter Logic Gaming  373.13     76646.0  360.0   532.0    903.0   
       Dignitas              423.73     86579.0  390.0   464.0    931.0   
       Evil Geniuses         605.52    121008.0  670.0   777.0   1571.0   

                             teamkills  ...  csdiffat10   goldat15     xpat15  \
season team                             ...                                     
2020   100 Thieves               534.0  ...       -99.0  1159872.0  1402835.0   
       Cloud9                    892.0  ...       824.0  1457462.0  1699932.0   
       Counter Logic Gaming      360.0  ...      -139.0   869714.0  1069385.0   
       Dignitas                  390.0  ...      -321.0  1007052.0  1219211.0   
       Evil Geniuses             670.0  ...      -499.0  1436032.0  1727591.0   

                              csat15  opp_goldat15  opp_xpat15  opp_csat15  \
season team                                                                  
2020   100 Thieves           24271.0     1196839.0   1427357.0     24539.0   
       Cloud9                29083.0     1339319.0   1616509.0     27782.0   
       Counter Logic Gaming  18978.0      926688.0   1108046.0     19352.0   
       Dignitas              21575.0     1028798.0   1237821.0     22083.0   
       Evil Geniuses         29739.0     1465996.0   1747335.0     30474.0   

                             golddiffat15  xpdiffat15  csdiffat15  
season team                                                        
2020   100 Thieves               -36967.0    -24522.0      -268.0  
       Cloud9                    118143.0     83423.0      1301.0  
       Counter Logic Gaming      -56974.0    -38661.0      -374.0  
       Dignitas                  -21746.0    -18610.0      -508.0  
       Evil Geniuses             -29964.0    -19744.0      -735.0  

[5 rows x 88 columns]

In [95]:
df['Win_Percentage'] = df['Wins'] / df['gamesplayed']

In [96]:
df.head(1000)

gamesplayed  Wins  Losses  Win_Percentage  \
season team                                                              
2020   100 Thieves                  49.0  20.0    29.0        0.408163   
       Cloud9                       57.0  44.0    13.0        0.771930   
       Counter Logic Gaming         37.0   8.0    29.0        0.216216   
       Dignitas                     42.0  15.0    27.0        0.357143   
       Evil Geniuses                60.0  29.0    31.0        0.483333   
       FlyQuest                     75.0  43.0    32.0        0.573333   
       Golden Guardians             51.0  23.0    28.0        0.450980   
       Immortals                    37.0  12.0    25.0        0.324324   
       Team Liquid                  49.0  29.0    20.0        0.591837   
       Team SoloMid                 71.0  41.0    30.0        0.577465   
2019   100 Thieves                  37.0  12.0    25.0        0.324324   
       Cloud9                       51.0  34.0    17.0        0.666667   
       Counter Logic Gaming         48.0  25.0    23.0        0.520833   
       FlyQuest                     45.0  18.0    27.0        0.400000   
       Golden Guardians             43.0  19.0    24.0        0.441860   
       Team Liquid                  54.0  40.0    14.0        0.740741   
       Team SoloMid                 54.0  32.0    22.0        0.592593   
2018   100 Thieves                  59.0  34.0    25.0        0.576271   
       Cloud9                       49.0  26.0    23.0        0.530612   
       Counter Logic Gaming         36.0  14.0    22.0        0.388889   
       FlyQuest                     41.0  16.0    25.0        0.390244   
       Golden Guardians             36.0   9.0    27.0        0.250000   
       Team Liquid                  55.0  39.0    16.0        0.709091   
       Team SoloMid                 59.0  33.0    26.0        0.559322   
2017   Cloud9                      101.0  65.0    36.0        0.643564   
       Counter Logic Gaming        106.0  57.0    49.0        0.537736   
       Dignitas                    109.0  55.0    54.0        0.504587   
       FlyQuest                    100.0  44.0    56.0        0.440000   
       Immortals                    96.0  55.0    41.0        0.572917   
       Team Liquid                 108.0  42.0    66.0        0.388889   
       Team SoloMid                104.0  73.0    31.0        0.701923   
2016   Cloud9                       79.0  48.0    31.0        0.607595   
       Counter Logic Gaming         86.0  49.0    37.0        0.569767   
       Dignitas                     21.0   4.0    17.0        0.190476   
       Immortals                    77.0  58.0    19.0        0.753247   
       Team Liquid                  76.0  38.0    38.0        0.500000   
       Team SoloMid                 78.0  58.0    20.0        0.743590   
2015   Cloud9                       47.0  24.0    23.0        0.510638   
       Counter Logic Gaming         47.0  31.0    16.0        0.659574   
       Dignitas                     44.0  19.0    25.0        0.431818   
       Team Liquid                  59.0  36.0    23.0        0.610169   
       Team SoloMid                 55.0  36.0    19.0        0.654545   
2014   Cloud9                       69.0  52.0    17.0        0.753623   
       Counter Logic Gaming         75.0  40.0    35.0        0.533333   
       Dignitas                     70.0  32.0    38.0        0.457143   
       Evil Geniuses                63.0  25.0    38.0        0.396825   
       Team SoloMid                 76.0  49.0    27.0        0.644737   

                              patch  gamelength   kills  deaths  assists  \
season team                                                                
2020   100 Thieves           494.06     98775.0   534.0   625.0   1213.0   
       Cloud9                575.34    106591.0   892.0   528.0   2073.0   
       Counter Logic Gaming  373.13     76646.0   360.0   532.0    903.0   
       Dignitas       

In [97]:
cols = list(df)
cols.insert(1, cols.pop(cols.index('Win_Percentage')))
df = df.loc[:, cols]
df

gamesplayed  Win_Percentage  Wins  Losses  \
season team                                                              
2020   100 Thieves                  49.0        0.408163  20.0    29.0   
       Cloud9                       57.0        0.771930  44.0    13.0   
       Counter Logic Gaming         37.0        0.216216   8.0    29.0   
       Dignitas                     42.0        0.357143  15.0    27.0   
       Evil Geniuses                60.0        0.483333  29.0    31.0   
       FlyQuest                     75.0        0.573333  43.0    32.0   
       Golden Guardians             51.0        0.450980  23.0    28.0   
       Immortals                    37.0        0.324324  12.0    25.0   
       Team Liquid                  49.0        0.591837  29.0    20.0   
       Team SoloMid                 71.0        0.577465  41.0    30.0   
2019   100 Thieves                  37.0        0.324324  12.0    25.0   
       Cloud9                       51.0        0.666667  34.0    17.0   
       Counter Logic Gaming         48.0        0.520833  25.0    23.0   
       FlyQuest                     45.0        0.400000  18.0    27.0   
       Golden Guardians             43.0        0.441860  19.0    24.0   
       Team Liquid                  54.0        0.740741  40.0    14.0   
       Team SoloMid                 54.0        0.592593  32.0    22.0   
2018   100 Thieves                  59.0        0.576271  34.0    25.0   
       Cloud9                       49.0        0.530612  26.0    23.0   
       Counter Logic Gaming         36.0        0.388889  14.0    22.0   
       FlyQuest                     41.0        0.390244  16.0    25.0   
       Golden Guardians             36.0        0.250000   9.0    27.0   
       Team Liquid                  55.0        0.709091  39.0    16.0   
       Team SoloMid                 59.0        0.559322  33.0    26.0   
2017   Cloud9                      101.0        0.643564  65.0    36.0   
       Counter Logic Gaming        106.0        0.537736  57.0    49.0   
       Dignitas                    109.0        0.504587  55.0    54.0   
       FlyQuest                    100.0        0.440000  44.0    56.0   
       Immortals                    96.0        0.572917  55.0    41.0   
       Team Liquid                 108.0        0.388889  42.0    66.0   
       Team SoloMid                104.0        0.701923  73.0    31.0   
2016   Cloud9                       79.0        0.607595  48.0    31.0   
       Counter Logic Gaming         86.0        0.569767  49.0    37.0   
       Dignitas                     21.0        0.190476   4.0    17.0   
       Immortals                    77.0        0.753247  58.0    19.0   
       Team Liquid                  76.0        0.500000  38.0    38.0   
       Team SoloMid                 78.0        0.743590  58.0    20.0   
2015   Cloud9                       47.0        0.510638  24.0    23.0   
       Counter Logic Gaming         47.0        0.659574  31.0    16.0   
       Dignitas                     44.0        0.431818  19.0    25.0   
       Team Liquid                  59.0        0.610169  36.0    23.0   
       Team SoloMid                 55.0        0.654545  36.0    19.0   
2014   Cloud9                       69.0        0.753623  52.0    17.0   
       Counter Logic Gaming         75.0        0.533333  40.0    35.0   
       Dignitas                     70.0        0.457143  32.0    38.0   
       Evil Geniuses                63.0        0.396825  25.0    38.0   
       Team SoloMid                 76.0        0.644737  49.0    27.0   

                              patch  gamelength   kills  deaths  assists  \
season team                                                                
2020   100 Thieves           494.06     98775.0   534.0   625.0   1213.0   
       Cloud9                575.34    106591.0   892.0   528.0   2073.0   
       Counter Logic Gaming  373.13     76646.0   360.0   532.0    903.0   
       Dignitas       

In [98]:
#eliminate any columns that have 0s
df = df.loc[:, (df != 0).any(axis=0)]

In [99]:
#export to CSV file to be used for EDA
df.to_csv(r'/Users/dankang/Desktop/SpringBoard/Capstone2/Capstone2/data/processed/Wrangled_LoL_Data.csv')

This dataframe is ready for EDA